# 获取视频信息数据

已有视频链接信息，获取点赞数，播放量，视频标签等详细信息，结果保存在 `../data/video/video_final.csv`

In [19]:
import pandas as pd
import numpy as np
import re
import requests
import json
import time
from tqdm import tqdm

In [20]:
df = pd.read_csv("../data/video/video.csv")
df = df.rename(columns={'标题_链接':'v_link', "标题": "v_name", "时间": "v_time", "图片": "v_img", "关键词": "v_dur", "关键词1": "v_key1", "数量": "v_num_play", "名称": "u_id", "名称_链接": "u_link"})
# 去除多余换行符
df_ori = df[['v_time', 'v_num_play']]
df["v_num_play"] = [re.compile('\n|\ ').sub("", df_ori["v_num_play"][i]) for i in range(df_ori.shape[0])]
df["v_time"] = [re.compile('\n|\ ').sub("", df_ori["v_time"][i]) for i in range(df_ori.shape[0])]
df["u_id"] = [re.findall(r'[1-9]+\.?[0-9]*',t)[0] for t in df['u_link']]
df["v_id"] = df.apply(lambda x: x['v_link'][31:43], axis = 1)

In [21]:
df.shape, df.drop_duplicates().shape

((961, 10), (961, 10))

In [22]:
df.to_csv("../data/video/video_clean_v1.csv", index = False, header=True)
df = pd.read_csv("../data/video/video_clean_v1.csv")

In [8]:
# 得到视频相关基本信息
def get_info_v(vid):
    time.sleep(np.random.uniform(low=0.1, high=0.5, size=1).item())
    payload={}
    headers = {}
    url="http://api.bilibili.com/x/web-interface/view?bvid={}".format(vid)
    response = requests.request("GET", url, headers=headers, data=payload)
    info=json.loads(response.text)
    info_stat = info['data']['stat']
    info_dic = {"v_id": vid,
                "aid": info_stat['aid'],
                "cid": info['data']['cid'],
                "reply": info_stat['reply'],
                "danmu": info_stat['danmaku'],
                "favorite": info_stat['favorite'],
                "coin": info_stat['coin'],
                "share": info_stat['share'],
                "like": info_stat['like'],
                "dislike": info_stat['dislike'],
                "tname": info['data']['tname']}
    return info_dic

---

In [ ]:
vid_list = df['v_id'].to_list()
info_add_list = []
for vid in tqdm(vid_list):
    try:
        info_add = get_info_v(vid)
        info_add_list.append(info_add)
    except:
        print(vid)
info_add = pd.DataFrame(info_add_list)

In [41]:
df = pd.concat([df, info_add], axis = 1)

In [23]:
# info_add.to_csv("../data/video_info.csv", index = False, header = True)
info_add = pd.read_csv("../data/video/video_info.csv")

In [33]:
# df = pd.concat([df, info_add], axis = 1)
df = df.merge(info_add, on = "v_id")

In [37]:
# 补变量 tag，和其他基本信息api不同，不可同时获取
vid_list = df['v_id']
tags = []
for i, vid in enumerate(vid_list):
    time.sleep(np.random.uniform(low=0.2, high=0.5, size=1).item())
    payload={}
    headers = {}
    url="https://api.bilibili.com/x/tag/archive/tags?aid={}".format(df['aid'][i])
    response = requests.request("GET", url, headers=headers, data=payload)
    try:
        info=json.loads(response.text)
        # 避免以逗号分割，以免和csv分隔符装车
        tags.append("*".join([t['tag_name'] for t in info['data']]))
    except:
        print(i, vid)
    if i % 50 == 0:
        print(i, end = "-")

0-50-100-150-200-250-300-350-400-450-500-550-600-650-700-750-800-850-900-950-

In [43]:
df['tag'] = tags
df['tag_num'] = [len(t.split("*")) for t in tags]
df.to_csv("../data/video/video_clean_v2.csv", index=False, header=True)